# Web scraping Otodom (rent)
The program that collects data from the Otodom website (https://www.otodom.pl/) and saves in a csv file and access to the data

In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install numpy
!pip install geopy
!pip install html5lib
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install psycopg2-binary

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import html5lib
import psycopg2

In [3]:
%load_ext sql

In [4]:
locator = Nominatim(user_agent="myGeocoder") 

## Creating a function that takes data from a page and cleans it so that fans can no longer have the right type

In [5]:
def get_data(link):
    try:
        html_data = requests.get(link)
        html_data.content
        soup = BeautifulSoup(html_data.content, 'html5lib')
    except Exception:
        return None

    try:
        rent_price = soup.find('strong',attrs={"data-cy":"adPageHeaderPrice"}).text
        rent_price = rent_price.replace("zł","").replace(" ","").replace(",", ".")
        rent_price = float(rent_price)
    except Exception:
        rent_price = np.NaN

    try:
        m2 = soup.find('div',attrs={"aria-label":"Powierzchnia"}).find_all('div')[1].text
        m2 = m2.replace("m²","").replace(",",".")
        m2 = float(m2)
    except Exception:
        m2 = np.NaN
     
    try:
        Deposit = soup.find('div',attrs={"aria-label":"Kaucja"}).find_all('div')[1].text
        Deposit = Deposit.replace("zł","").replace(" ","").replace(",", ".")
        Deposit = float(Deposit)
    except Exception:
        Deposit = np.NaN

    try:
        Type_of_building = soup.find('div',attrs={"aria-label":"Rodzaj zabudowy"}).find_all('div')[1].text
        str(Type_of_building)
    except Exception:
        Type_of_building = np.NaN

    try:
        room_number = soup.find('div',attrs={"aria-label":"Liczba pokoi"}).find_all('div')[1].text
        room_number = room_number.replace(",", ".")
        room_number = float(room_number)
    except Exception:
        room_number = np.NaN

    try:
        Building_material = soup.find('div',attrs={"aria-label":"Materiał budynku"}).find_all('div')[1].text
        Building_material = str(Building_material)
    except Exception:
        Building_material = np.NaN
        
    try:
        Year_of_construction = soup.find('div',attrs={"aria-label":"Rok budowy"}).find_all('div')[1].text
        Year_of_construction = Year_of_construction.replace(",", ".")
        Year_of_construction = float(Year_of_construction)
    except Exception:
        Year_of_construction = np.NaN
    
    try:
        floor = soup.find('div',attrs={"aria-label":"Piętro"}).find_all('div')[1].text
        floor = floor.replace(",", ".")
        floor = float(floor)
    except Exception:
        floor = np.NaN
        
    try:
        windows = soup.find('div',attrs={"aria-label":"Okna"}).find_all('div')[1].text
        windows = str(windows)
    except Exception:
        windows = np.NaN
    
    try:
        Number_of_floors = soup.find('div',attrs={"aria-label":"Liczba pięter"}).find_all('div')[1].text
        Number_of_floors = Number_of_floors.replace(",", ".")
        Number_of_floors = float(Number_of_floors)
    except Exception:
        Number_of_floors = np.NaN
        
    try:
        Heating = soup.find('div',attrs={"aria-label":"Ogrzewanie"}).find_all('div')[1].text
        Heating = str(Heating)
    except Exception:
        Heating = np.NaN

    media = np.NaN
    tab_all = soup.find('div',attrs={"class":"css-1lw3ul3 ex3yvbv4"})
    try:
        for two in tab_all.find_all('div'):
            tab_all_deep = two.find_all('h3')[0].text
            if tab_all_deep == "media":
                tab_all_deep_2 = two.find_all('ul')[0]
                media = ""
                for one in tab_all_deep_2.find_all('li'):
                    media += one.text + ", "
                media = media[:-2]
    except:
        pass
     
    security = np.NaN
    tab_all = soup.find('div',attrs={"class":"css-1lw3ul3 ex3yvbv4"})
    try:
        for two in tab_all.find_all('div'):
            tab_all_deep = two.find_all('h3')[0].text
            if tab_all_deep == "zabezpieczenia":
                tab_all_deep_2 = two.find_all('ul')[0]
                security = ""
                for one in tab_all_deep_2.find_all('li'):
                    security += one.text + ", "
                security = security[:-2]
    except:
        pass

        
    Additional_information = np.NaN
    tab_all = soup.find('div',attrs={"class":"css-1lw3ul3 ex3yvbv4"})
    try:
        for two in tab_all.find_all('div'):
            tab_all_deep = two.find_all('h3')[0].text
            if tab_all_deep == "informacje dodatkowe":
                tab_all_deep_2 = two.find_all('ul')[0]
                Additional_information = ""
                for one in tab_all_deep_2.find_all('li'):
                    Additional_information += one.text + ", "
                Additional_information = Additional_information[:-2]
    except:
        pass

         
    Equipment = np.NaN
    tab_all = soup.find('div',attrs={"class":"css-1lw3ul3 ex3yvbv4"})
    try:
        for two in tab_all.find_all('div'):
            tab_all_deep = two.find_all('h3')[0].text
            if tab_all_deep == "wyposażenie":
                tab_all_deep_2 = two.find_all('ul')[0]
                Equipment = ""
                for one in tab_all_deep_2.find_all('li'):
                    Equipment += one.text + ", "
                Equipment = Equipment[:-2]
    except:
        pass

    try:
        link_2 = link
        link_2 = str(link_2)
    except Exception:
        pass
    
    try:
        location = soup.find('a',attrs={"class":"css-1ibwe9h e1nbpvi62"}).text
    except Exception:
        location = np.NaN
        
    # Generate latitude and longitude using location
    
    try:
        location = locator.geocode(location)
        geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
        latitude = location.latitude
        longitude = location.longitude
    except Exception:
        latitude = np.NaN
        longitude = np.NaN
    

        
    return {'rent_price': rent_price, 'm2': m2,'Deposit': Deposit, 'Type_of_building': Type_of_building,
           'room_number':room_number,'Building_material': Building_material, 'Year_of_construction': Year_of_construction,
            'floor': floor, 'windows': windows,
           'Number_of_floors': Number_of_floors, 'Heating': Heating,
            'media': media,'link_2': link_2, 'security': security, 'Additional_information': Additional_information,
            'Equipment': Equipment, 'location': location, 'latitude': latitude, 'longitude': longitude}

## Creating a table

In [6]:
table = {'rent_price': [], 'm2': [],'Deposit': [], 'Type_of_building': [],
        'room_number': [],'Building_material': [], 'Year_of_construction': [],
        'floor': [], 'windows': [],
         'Number_of_floors': [], 'Heating': [], 'media': [], 'security': [], 'link_2': [],
         'Additional_information': [], 'Equipment': [], 'location': [], 'latitude': [], 'longitude': []}

## Replace the table in a DataFrame

In [7]:
df_table = pd.DataFrame(table)
print(df_table)

Empty DataFrame
Columns: [rent_price, m2, Deposit, Type_of_building, room_number, Building_material, Year_of_construction, floor, windows, Number_of_floors, Heating, media, security, link_2, Additional_information, Equipment, location, latitude, longitude]
Index: []


## Main loop
The main loop where apartments for rent from Warsaw are selected.

First, the first page with apartments is opened and then the loop looks through all the mixed areas of this page and retrieves key information from themThen it goes to the next page.
Depending on the settings, we will help you browse a defined number of pages or all.

If we want to view all pages we use "while" and if a specific number of pages we enter it in "range" in the first "for" pentle

In [8]:
Link_ERROR = 0
Read_announcements = 0
page = 1
while True:
    
# for page in range(1,10):  
    try:
        html_data_link = requests.get(f"https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa?page={page}")
        page += 1
    except Exception:
        print("Link_ERROR")
        continue
    
    soup_link = BeautifulSoup(html_data_link.content, 'html5lib')
    
    if soup_link.find('div',attrs={"class":"css-pi2gf8 e1qwpsp45"}) is None:
        pass
    else:
        break

    for offer in soup_link.find_all('a',attrs={"data-cy":"listing-item-link"}):
            new_row = get_data("https://www.otodom.pl"+ offer['href'])
            
            
            if new_row == None:
                Link_ERROR += 1
            else:
                df_table = df_table.append(new_row, ignore_index=True)
                Read_announcements += 1
                clear_output(wait=True)
                print("Read_announcements ", Read_announcements)
                print("Link_ERROR ",Link_ERROR)     
                
print("Finish")

Read_announcements  7731
Link_ERROR  0
Finish


## We choose the name of the table and database

In [9]:
table_name = 'Warsaw_rent_full'

## Serving to the CSV file

In [10]:
df_table.to_csv (table_name +'.csv', index = False, header=True)

## Database connection

In [11]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otodom', user='barto', password='biznes')
cur = psql.cursor()

## Creating a table in the database

In [12]:
cur.execute('DROP TABLE IF EXISTS '+ table_name)
createTableCommand = 'CREATE TABLE ' + table_name + '''(ID int NOT NULL,
    rent_price float,
    m2 float,
    Deposit float,
    Type_of_building VARCHAR,
    room_number float,
    Building_material VARCHAR,
    Year_of_construction float,
    floor float,
    windows VARCHAR,
    Number_of_floors float,
    Heating VARCHAR,
    media VARCHAR,
    security VARCHAR,
    link_2 VARCHAR,
    Additional_information VARCHAR,
    Equipment VARCHAR,
    location VARCHAR,
    latitude float,
    longitude float,
    PRIMARY KEY (ID)
    )'''
cur.execute(createTableCommand)
psql.commit()

## Loading data into the database

In [13]:
insert_table = ['ID',
    'rent_price',
    'm2',
    'Deposit',
    'Type_of_building',
    'room_number',
    'Building_material',
    'Year_of_construction',
    'floor',
    'windows',
    'Number_of_floors',
    'Heating',
    'media',
    'security',
    'link_2',
    'Additional_information',
    'Equipment',
    'location',
    'latitude',
    'longitude']


a = 0
for row in df_table.itertuples():
    full=""
    full_fit="INSERT INTO " + table_name + "("
    for (column, insert) in zip(row, insert_table):
        if str(column) =='nan':
            continue
        column = str(column)
        column = column.replace("'", "''")
        full +="'"+column+"'"+", "
        full_fit += insert + ', '  
        
    full=full[:-2]
    full_fit=full_fit[:-2]
    full_fit=full_fit+')'
    Value_data='VALUES ('+full+');'
    insertDataCommand = full_fit+"\r\n"+Value_data
    cur.execute(insertDataCommand)
psql.commit()
print("Finish")

Finish
